In [12]:
import autogen
from autogen import ConversableAgent

In [13]:
config_list = autogen.config_list_from_json(
    env_or_file="AOAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": {
            "gpt-4o"
        }
    },
)

In [14]:
initializer = autogen.UserProxyAgent(
    name="Init",
    code_execution_config={
        "use_docker": False,
    }
)

In [15]:
coder = autogen.AssistantAgent(
    name="Retrieve_Action_1",
    llm_config= { "config_list": config_list },
    system_message="""
    You are the Coder. Given a topic, write code to retrieve related papers from the arXiv API, 
    print their title, authors, abstract, and link.
    You write python/shell code to solve tasks. Wrap the code in a code block that specifies 
    the script type. The user can't modify your code. So do not suggest incomplete code which 
    requires others to modify. Don't use a code block if it's not intended to be executed by 
    the executor. Don't include multiple code blocks in one response. 
    Do not ask others to copy and paste the result. Check the execution result returned by 
    the executor. If the result indicates there is an error, fix the error and output the code again. 
    Suggest the full code instead of partial code or code changes. 
    If the error can't be fixed or if the task is not solved even after the code is executed successfully, 
    analyze the problem, revisit your assumption, collect additional info you need, and think of a 
    different approach to try.
    """,
)

In [16]:
executor = autogen.UserProxyAgent(
    name="Retrieve_Action_2",
    system_message="Executor. Execute the code written by the Coder and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

In [17]:
scientist = autogen.AssistantAgent(
    name="Research_Action_1",
    llm_config={ "config_list": config_list },
    system_message="""
    You are the Scientist. Please categorize papers after seeing their abstracts printed 
    and create a markdown table with Domain, Title, Authors, Summary and Link
    """
)

Below is the function to select the speaker deterministically as per need 

In [38]:
import json
from autogen import Agent, GroupChat

def state_transition(last_speaker: Agent, groupchat: GroupChat):
    messages = groupchat.messages
    print(f"Message: {json.dumps(messages)}")
    
    if last_speaker is initializer:
        # init -> retrieve
        return coder
    elif last_speaker is coder:
        # retrieve: action 1 -> action 2
        return executor
    elif last_speaker is executor:
        if "exitcode: 1" in messages[-1]["content"]:
            # retrieve --(execution failed)--> retrieve
            return coder
        else:
            # retrieve --(execution success)--> research
            return scientist
    elif last_speaker == "Scientist":
        # research -> end
        return None

In [39]:
groupchat = autogen.GroupChat(
    agents=[initializer, coder, executor, scientist],
    messages=[],
    max_round=20,
    speaker_selection_method=state_transition,
)

In [40]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, 
    llm_config={ "config_list": config_list }
)

In [41]:
initializer.initiate_chat(
    manager, 
    message="Topic: LLM applications papers from last week. Requirement: 5 - 10 papers from different domains."
)

Init (to chat_manager):

Topic: LLM applications papers from last week. Requirement: 5 - 10 papers from different domains.

--------------------------------------------------------------------------------
Message: [{"content": "Topic: LLM applications papers from last week. Requirement: 5 - 10 papers from different domains.", "role": "user", "name": "Init"}]

Next speaker: Retrieve_Action_1

Retrieve_Action_1 (to chat_manager):

Sure, I'll write a Python script that uses the arXiv API to retrieve papers on the topic of "LLM applications" from the past week. This script will print the title, authors, abstract, and link for each paper. 

```python
import requests
import datetime
from dateutil import parser

# Fetching current date and date of a week before
current_date = datetime.datetime.now()
last_week_date = current_date - datetime.timedelta(weeks=1)

# Format dates in the required format for arXiv API
current_date_str = current_date.strftime("%Y%m%d")
last_week_date_str = last_week_d

No eligible speaker found. Terminating the conversation.


Message: [{"content": "Topic: LLM applications papers from last week. Requirement: 5 - 10 papers from different domains.", "role": "user", "name": "Init"}, {"content": "Sure, I'll write a Python script that uses the arXiv API to retrieve papers on the topic of \"LLM applications\" from the past week. This script will print the title, authors, abstract, and link for each paper. \n\n```python\nimport requests\nimport datetime\nfrom dateutil import parser\n\n# Fetching current date and date of a week before\ncurrent_date = datetime.datetime.now()\nlast_week_date = current_date - datetime.timedelta(weeks=1)\n\n# Format dates in the required format for arXiv API\ncurrent_date_str = current_date.strftime(\"%Y%m%d\")\nlast_week_date_str = last_week_date.strftime(\"%Y%m%d\")\n\n# Parameters for the arXiv API request\nsearch_query = 'all:LLM applications'\nmax_results = 10\n\n# URL for the arXiv API request\nurl = f'http://export.arxiv.org/api/query?search_query={search_query}&start=0&max_resul

ChatResult(chat_id=None, chat_history=[{'content': 'Topic: LLM applications papers from last week. Requirement: 5 - 10 papers from different domains.', 'role': 'assistant', 'name': 'Init'}, {'content': 'Sure, I\'ll write a Python script that uses the arXiv API to retrieve papers on the topic of "LLM applications" from the past week. This script will print the title, authors, abstract, and link for each paper. \n\n```python\nimport requests\nimport datetime\nfrom dateutil import parser\n\n# Fetching current date and date of a week before\ncurrent_date = datetime.datetime.now()\nlast_week_date = current_date - datetime.timedelta(weeks=1)\n\n# Format dates in the required format for arXiv API\ncurrent_date_str = current_date.strftime("%Y%m%d")\nlast_week_date_str = last_week_date.strftime("%Y%m%d")\n\n# Parameters for the arXiv API request\nsearch_query = \'all:LLM applications\'\nmax_results = 10\n\n# URL for the arXiv API request\nurl = f\'http://export.arxiv.org/api/query?search_query=